In [1]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import h5py
import pandas as pd
%matplotlib inline

os.chdir('/hgst8TB/fjurisic/ecoli')
os.listdir('.')

['test.txt',
 'mapping_errors.log',
 'pass',
 'reference2.fasta',
 'ref',
 'refstring2.txt',
 'train.txt']

In [2]:
with open('./train.txt') as f:
    print(f.readline())

26075_ch100_read13_strand	1899



In [3]:
os.listdir('./pass')

['40525_ch500_read242_strand.fast5',
 '26075_ch40_read78_strand.fast5',
 '66703_ch387_read16_strand.fast5',
 '40525_ch240_read163_strand.fast5',
 '40525_ch353_read220_strand.fast5',
 '66703_ch320_read21_strand.fast5',
 '40525_ch181_read82_strand.fast5',
 '95274_ch306_read763_strand.fast5',
 '26075_ch248_read555_strand.fast5',
 '26075_ch471_read118_strand1.fast5',
 '66703_ch158_read468_strand.fast5',
 '40525_ch512_read420_strand.fast5',
 '66703_ch215_read802_strand.fast5',
 '66703_ch230_read203_strand.fast5',
 '66703_ch246_read470_strand.fast5',
 '40525_ch182_read519_strand.fast5',
 '66703_ch448_read212_strand.fast5',
 '40525_ch103_read59_strand.fast5',
 '66703_ch445_read42_strand.fast5',
 '95274_ch470_read48302_strand.fast5',
 '26075_ch296_read80_strand.fast5',
 '95274_ch414_read479_strand.fast5',
 '26075_ch505_read131_strand.fast5',
 '40525_ch223_read8_strand.fast5',
 '95274_ch13_read182_strand.fast5',
 '40525_ch315_read489_strand.fast5',
 '38623_ch246_read113_strand.fast5',
 '26075_c

In [4]:
h5 = h5py.File('./pass/40525_ch500_read242_strand.fast5', 'r')

In [5]:
reads = h5['Analyses/EventDetection_000/Reads/']
print(reads.keys().__len__())
for x in reads:
    ds = reads[x]['Events']

1


In [ ]:
sns.pairplot(data=df, kind='reg', vars=['mean', 'variance', 'length'])

In [ ]:
with open('./ref/40525_ch500_read242_strand.ref') as f:
    gg = f.readlines()
    print (gg[-1])

In [ ]:
df = []
for line in open('./train.txt'):
    df.append(line.split())
df = pd.DataFrame.from_records(df, columns=['fname', 'len'])

In [ ]:
df['len'] = df.len.astype(np.int32)

In [ ]:
sns.boxplot(df['len'])

In [10]:
def small_train(n = 10, max_base_pairs = 100):
    with open('./train.txt') as f:
        lines = f.readlines()
    df = []
    for line in open('./train.txt'):
        df.append(line.split())
    df = pd.DataFrame.from_records(df, columns=['fname', 'len'])
    df['len'] = df.len.astype(np.int32)
    df = df[df['len'] > max_base_pairs]
    df = df.sample(frac=1)

    sol = {}
    for fname in df[:n]['fname']:
        sol[fname] = {}
        with h5py.File('./pass/%s.fast5' % fname, 'r') as h5, open('./ref/%s.ref' % fname) as f:
            target = f.readlines()[-1].strip() ## line without \n
            target = list(map(lambda x: {"A":0, "G":1, "T":2, "C":3}[x], target))
            
            reads = h5['Analyses/EventDetection_000/Reads/']
            assert reads.keys().__len__() == 1
            for x in reads.values():
                ds = np.array(x['Events'])
                source = pd.DataFrame(ds, columns=['length', 'mean', 'stdv']) 
        
        source = source[:int(len(source)/len(target)*max_base_pairs)]
        target = target[:max_base_pairs]
        
        sol[fname]['source'] = source
        sol[fname]['source_len'] = len(source)
        sol[fname]['target'] = target
        sol[fname]['target_len'] = len(target)
    return sol
    
ff = small_train()
for gg in ff.values():
    print(gg['source_len'], gg['target_len'], gg['source_len']/gg['target_len'])

163 100 1.63
198 100 1.98
143 100 1.43
188 100 1.88
193 100 1.93
174 100 1.74
168 100 1.68
182 100 1.82
160 100 1.6
194 100 1.94


In [11]:
def small_train_processed(n = 10, max_base_pairs = 100):
    dataset = small_train(n, max_base_pairs)
    
    dval = list(dataset.values())  # To enforce ordering
    
    x_len = np.array([x['source_len'] for x in dval])
    x = np.zeros((n, np.max(x_len), 3), dtype=np.float32)
    y_len = np.array([x['target_len'] for x in dval])
    y = np.zeros((n, np.max(y_len)))
    assert np.max(y_len) == max_base_pairs
    
    for i, d in enumerate(dval):
        x_len[i] = d['source_len']
        x[i, :x_len[i]] = d['source']
        
        y_len[i] = d['target_len']
        y[i, :y_len[i]] = d['target']

    return x, x_len, y, y_len
x, x_len, y, y_len = small_train_processed()

In [16]:
import tensorflow as tf

with tf.Session() as sess:
    tf.reset_default_graph()
    X = tf.constant(x)
    X_len  = tf.constant(x_len)
    Y = tf.constant(y)
    Y_len = tf.constant(y_len)

    batch_size = x.shape[0]
    state_size = 100

    cell = tf.nn.rnn_cell.GRUCell(state_size)
    init_state = cell.zero_state(batch_size, tf.float32)
#     init_state = tf.tile(init_state, [batch_size, 1])

#     rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, X, sequence_length=X_len, initial_state=init_state)
    
    sess.run(tf.initialize_all_variables())

IndexError: list index out of range

In [ ]:
a = sess.run(init_state)


In [ ]:
a = sess.run(rnn_outputs)
a.shape, x.shape

In [ ]:
idx = tf.range(batch_size)*tf.shape(rnn_outputs)[1] + (X_len - 1)
last_rnn_output = tf.gather(tf.reshape(rnn_outputs, [-1, state_size]), idx)